In [1]:
# python3 -m jupyterlab --no-browser --ip=0.0.0.0 --port=5050 --allow-root --NotebookApp.token='local'
import json
import transformers
import time
import logging
import os
import wandb
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import datasets
import numpy as np
from datasets import load_dataset, load_metric
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from fly_src.models.modeling_roberta import RobertaForSequenceClassification
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from train_utils import *


[2023-12-29 16:22:00,577] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [9]:
lora_dict = torch.load("results/lora/cola/model/checkpoint-13547/pytorch_model.bin")
for name, module in lora_dict.items():
    if "lora" in name:
        print(name, ",", module.shape)

roberta.encoder.layer.0.attention.self.query.lora_A , torch.Size([8, 1024])
roberta.encoder.layer.0.attention.self.query.lora_B , torch.Size([1024, 8])
roberta.encoder.layer.0.attention.self.value.lora_A , torch.Size([8, 1024])
roberta.encoder.layer.0.attention.self.value.lora_B , torch.Size([1024, 8])
roberta.encoder.layer.1.attention.self.query.lora_A , torch.Size([8, 1024])
roberta.encoder.layer.1.attention.self.query.lora_B , torch.Size([1024, 8])
roberta.encoder.layer.1.attention.self.value.lora_A , torch.Size([8, 1024])
roberta.encoder.layer.1.attention.self.value.lora_B , torch.Size([1024, 8])
roberta.encoder.layer.2.attention.self.query.lora_A , torch.Size([8, 1024])
roberta.encoder.layer.2.attention.self.query.lora_B , torch.Size([1024, 8])
roberta.encoder.layer.2.attention.self.value.lora_A , torch.Size([8, 1024])
roberta.encoder.layer.2.attention.self.value.lora_B , torch.Size([1024, 8])
roberta.encoder.layer.3.attention.self.query.lora_A , torch.Size([8, 1024])
roberta.enco

In [14]:
monarch_dict = torch.load("results/monarch_roberta_glue/cola/checkpoint-2250/pytorch_model.bin")
for name, module in monarch_dict.items():
    if "key" in name:
        print(name, ",", module.shape)


roberta.encoder.layer.0.attention.self.key.bias , torch.Size([1024])
roberta.encoder.layer.0.attention.self.key.dense , torch.Size([1024, 1024])
roberta.encoder.layer.1.attention.self.key.bias , torch.Size([1024])
roberta.encoder.layer.1.attention.self.key.dense , torch.Size([1024, 1024])
roberta.encoder.layer.2.attention.self.key.bias , torch.Size([1024])
roberta.encoder.layer.2.attention.self.key.dense , torch.Size([1024, 1024])
roberta.encoder.layer.3.attention.self.key.bias , torch.Size([1024])
roberta.encoder.layer.3.attention.self.key.dense , torch.Size([1024, 1024])
roberta.encoder.layer.4.attention.self.key.bias , torch.Size([1024])
roberta.encoder.layer.4.attention.self.key.dense , torch.Size([1024, 1024])
roberta.encoder.layer.5.attention.self.key.bias , torch.Size([1024])
roberta.encoder.layer.5.attention.self.key.dense , torch.Size([1024, 1024])
roberta.encoder.layer.6.attention.self.key.bias , torch.Size([1024])
roberta.encoder.layer.6.attention.self.key.dense , torch.Size

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torch

def hook_fn(module, grad_input, grad_output):
    print(f"{module} has dW {grad_input[1]} and scaler value {module.scaler}")
    
    
def factor_balance(mid_blksz, out_blksz):
    total = mid_blksz * out_blksz

class Scaler(nn.Module):
    def __init__(self, out_features):
        super().__init__()
        self.scaler = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        # x.requires_grad_(True)
        x = self.scaler * x
        # layernorm to avoid vanishing gradient
        return x

x = torch.ones(100, 100, dtype=torch.float32)
y = torch.full((100, 100), 2, dtype=torch.float32)
model = Scaler(100)
model.register_backward_hook(hook_fn)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
for i in range(100):
    loss = F.mse_loss(model(x), y)
    loss.backward()
    optimizer.step()
    

Scaler() has dW None and scaler value Parameter containing:
tensor([0.], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([0.4000], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([1.1200], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([2.0160], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([2.9088], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([3.6198], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([4.0069], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([3.9926], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([3.5798], requires_grad=True)
Scaler() has dW None and scaler value Parameter containing:
tensor([2.8510], requires_grad=True)
Scaler() has dW None and scaler va